# How Python Manages Memory #

Python is a *dynamic* language. Not only do you not need to declare the types of variables, function results/arguments and so on, but for the most part you don’t even need to worry about allocating and freeing memory for objects—Python manages that automatically.

Allocating memory is easy, but when do you free up the memory? How does Python know when you have finished with an object, so the storage for it can be reclaimed?

There are two common techniques for keeping track of memory allocations:
* *Reference counting*, where the number of variables or other objects that refer to this one is stored in a field in the object. This is incremented every time a new reference to this object is stored somewhere, and decremented every time such a reference is removed (whether because it has been replaced with a reference to something else, or the containing object has been reclaimed). When this goes to zero, the object can be deleted and its storage reused.
* *Garbage collection*, which scans through all the variables in your program, looking for what objects they hold references to, and what other objects those in turn hold references to, and so on. Any objects for which storage is allocated, but which are not encountered in such a scan, can be deallocated.

CPython (which is the Python implementation in most common use, the one maintained by the Python Software Foundation and available from `python.org`, and is the one you’re using now in this notebook) uses both techniques.

The following demonstrates how each object has a reference count attached, which can be accessed with the `sys.getrefcount()` function:

In [ ]:
import sys

class MyObj :

    def __init__(self, name) :
        self.name = name
    #end __init__

    def __repr__(self) :
        return "%s(%s)" % (type(self).__name__, repr(self.name))
    #end __repr__

#end MyObj

a = MyObj("a")
print("initial refcount:", sys.getrefcount(a))
b = a
print("after assigning to b:", sys.getrefcount(a))
b = None
print("after deassigning from b:", sys.getrefcount(a))


Why is the initial refcount 2 and not 1? Probably because `sys.getrefcount()`, like any normal function, needs to have another reference to the object in its function argument. Here is what the reference count looks like for an object which is created only to be passed to `sys.getrefcount()`, and then discarded:

In [ ]:
print("refcount for throwaway object:", sys.getrefcount(MyObj("throwaway")))

Besides the normal kind of object reference you get when you assign an object to a variable as above, there is another kind of object reference, called a *weak* reference. These are basically saying “I only want to keep this reference as long as the object actually exists, but I don’t want my reference to count towards the reasons for not deleting it”. As soon as the object disappears (because all the regular kind of references have disappeared), then any weak references to that object will also automatically disappear.

Weak references have many uses, one of which is as a diagnostic tool. Here we obtain a weak reference to the object above, and as soon as the last reference to this object is overwritten, the object disappears. This uses the `weakref.ref()` function, which creates a callable that will return a strong reference to the object as long as it exists, otherwise it returns (a strong reference to) `None`:


In [ ]:
import weakref

c = weakref.ref(a)
print("before nulling a:", c())
a = None
print("after nulling a:", c())


Why does CPython use both memory-management techniques? The reason for this is that the two techniques have pros and cons.

As you can see above, reference counting usually frees up storage quickly. If an object is only assigned to one variable, its reference count will be 1. As soon as the variable is assigned something else (or goes out of scope), the object reference count goes to 0 and the object can be freed.

However, it is possible to have *reference cycles*, where object A contains a reference to object B, while object B in turn contains a reference to object A (longer chains with more than two such links are also possible); in this case, even when nothing else holds any references to either object, they each still have a reference count of 1, and so reference counting alone will not determine that they can both be freed.

The following example uses a `WeakSet`, which every time it is examined will be seen to contain (strong) references to members which still exist. As those members disappear, they will also disappear from the set the next time it is examined.

In [ ]:
a = MyObj("a")
b = MyObj("b")
c = MyObj("c")
a.link = b
b.link = a
x = weakref.WeakSet((a, b, c))
print("before losing strong refs:", list(x))
a = b = c = None
print("after losing strong refs:", list(x))


As you can see, while the object in `c` has disappeared, even after variables `a` and `b` have lost their references to the other two interlinked objects, they haven’t gone away.

The only way to get rid of such “orphaned” objects is via garbage collection. This will often happen automatically during the run of a Python program, but it is possible to force it to run:

In [ ]:
import gc

gc.collect()
print("after gc:", list(x))


A further disadvantage of reference-counting is the infamous *Global Interpreter Lock* (“GIL”) in the CPython implementation. What this means is that the CPython interpreter cannot multithread references to objects. In order to ensure that the reference counts are correctly updated, other threads have to be locked out of the interpreter, to prevent them from accessing any Python objects at the same time. So pure-Python code cannot make use of modern multi-CPU hardware, at least when running under CPython, since only one thread will be able to run at any one time!

Garbage collection deals correctly with reference cycles, and it has no problem with multithreaded interpreters. So why not use it all the time, and dispense with reference-counting? The big problem is deciding when to run it. Running it frequently adds to CPU overhead. But then, running it infrequently means the program’s memory usage might grow; even though it does not need a lot of memory for live objects at any particular time, it might be allocating new objects (and forgetting old ones) at a high rate, so these will be hanging around, consuming excess memory, in-between garbage-collection runs.

Some languages that rely purely on garbage collection for memory management, like Java, have to impose a limit on maximum process memory size, in order to keep the program from gobbling up all available system memory. Then this becomes another configuration chore: how big do you make the memory size? You have to decide how much is needed for “real” memory usage by objects with significant lifetimes, instead of letting the program automatically ask the system for how much it needs.

CPython gives you the choice. You can rely on garbage collection if you want. Or, if you write your code carefully enough, you can exploit its use of reference-counting to keep your program’s memory usage down.

## Suddenly Cycles ##

Reference cycles can often be introduced in subtle ways. Consider the following very simple class, which allows the caller to attach callbacks which are invoked in response to a particular method call:

In [ ]:
class MessageHandler :

    def __init__(self) :
        self.handlers = []
    #end __init__

    def send(self, val) :
        for h in self.handlers :
            h[0](h[1], val)
        #end for
    #end send

#end MessageHandler

Looks pretty straightforward so far? Here is an example use of this class:

In [ ]:
def my_handle1(obj, val) :
    print("%s got val = %s" % (repr(obj), repr(val)))
#end my_handle1

mh = MessageHandler()
mh.handlers.append((my_handle1, "some val"))
mh.send("hello 1")

Now, let’s check that discarding the only known reference to our `MessageHandler` object causes it to disappear:

In [ ]:
w_mh = weakref.ref(mh)
print("before nulling:", w_mh())
mh = None
print("after nulling:", w_mh())

Yup, looks good. Let’s try another use of the same class, this time passing a reference to the object itself to the callback as the first argument:

In [ ]:
mh = MessageHandler()
mh.handlers.append((my_handle1, mh))
mh.send("hello 2")

But watch what happens when I try to make the `MessageHandler` object go away:

In [ ]:
w_mh = weakref.ref(mh)
print("before nulling:", w_mh())
mh = None
print("after nulling:", w_mh())

It doesn’t go away! Why not? Can you see the reference cycle?

    mh → mh.handlers → [my_handle1, mh]

How do we fix this? We need to break the reference cycle. How? By finding another use for weak refs, substituting one of them in place of one of the strong refs in the above chain.
The obvious place is the parent object in the callback entry:

In [ ]:
def my_handle2(w_obj, val) :
    obj = w_obj()
    assert obj != None
    print("%s got val = %s" % (repr(obj), repr(val)))
#end my_handle2

mh = MessageHandler()
mh.handlers.append((my_handle2, weakref.ref(mh)))
mh.send("hello 2")

As you can see, the output looks exactly the same as before.

Does the `MessageHandler` now correctly go away?

In [ ]:
w_mh = weakref.ref(mh)
print("before nulling:", w_mh())
mh = None
print("after nulling:", w_mh())

Yes, it does!

## To Garbage Collect, Or Not To Garbage Collect? ##

Given some of the intricacies in avoiding cycles with reference counting, it seems tempting to give up and simply fall back on garbage collection all the time. As mentioned, some languages (and some alternative Python implementations) already take this approach. But that doesn’t get you completely out of the woods: besides the performance and resource-usage issues already discussed, it is still possible to get memory leaks, even in a purely garbage-collected language. And they can be just as tricky to track down.

But that discussion is for another day.